<a href="https://colab.research.google.com/github/wnydng/Hackathon-BI-Pipeline/blob/safa/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json, os, re
from sklearn.model_selection import train_test_split

# ============ Helpers ============
CURRENCY_RE = re.compile(r"[^\d\-\.\,]")

def clean_currency(val):
    """Convertit '$1,234.56' ou '€-1.234,56' -> float"""
    if pd.isna(val):
        return np.nan
    if isinstance(val, (int, float, np.integer, np.floating)):
        return float(val)
    s = str(val).strip()
    s = CURRENCY_RE.sub("", s)  # garde chiffres, ., ,, -
    if "," in s and "." in s:
        # format EU: '.' milliers, ',' décimal
        s = s.replace(".", "").replace(",", ".")
    elif "," in s:
        parts = s.split(",")
        s = ".".join(parts) if len(parts[-1]) != 3 else "".join(parts)
    try:
        return float(s)
    except:
        return np.nan

def to_zip_str(z):
    """Conserve les zéros en tête quand zip est numérique."""
    if pd.isna(z):
        return pd.NA
    s = str(z).strip()
    if s.replace(".","",1).isdigit():
        try:
            return f"{int(float(s)):05d}"
        except:
            return s
    return s

def parse_dt(x):
    try:
        return pd.to_datetime(x, errors="coerce")
    except:
        return pd.NaT

def normalize_str(x):
    return pd.NA if pd.isna(x) else str(x).strip()

def infer_city_from_address(addr):
    """Heuristique simple: prend le morceau après la dernière virgule s'il contient des lettres."""
    if pd.isna(addr):
        return pd.NA
    parts = [p.strip() for p in str(addr).split(",")]
    if len(parts) < 2:
        return pd.NA
    cand = parts[-1]
    return cand if re.search(r"[A-Za-z]", cand) else pd.NA

# ============ Preps ============
def prep_transactions(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "amount" in df: df["amount"] = df["amount"].apply(clean_currency)
    if "date" in df:   df["date"]   = df["date"].apply(parse_dt)
    if "zip" in df:    df["zip"]    = df["zip"].apply(to_zip_str)
    for c in ["merchant_city", "merchant_state"]:
        if c in df: df[c] = df[c].apply(normalize_str)
    if "use_chip" in df:
        df["use_chip"] = df["use_chip"].map({
            "Chip Transaction": "chip",
            "Swipe Transaction": "swipe",
            "Online Transaction": "online"
        }).fillna(df["use_chip"].astype(str).str.lower())
    if "mcc" in df: df["mcc"] = df["mcc"].astype(str).str.strip()
    for c in ["transaction_id", "client_id", "card_id", "merchant_id"]:
        if c in df: df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")
    # ville = merchant_city en priorité
    if "merchant_city" in df: df["city"] = df["merchant_city"]
    return df

def prep_cards(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "credit_limit" in df:   df["credit_limit"]   = df["credit_limit"].apply(clean_currency)
    if "acct_open_date" in df: df["acct_open_date"] = pd.to_datetime(df["acct_open_date"], errors="coerce")
    if "id" in df:
        df["id"] = pd.to_numeric(df["id"], errors="coerce").astype("Int64")
        # déduplication: si dispo, garde l'enregistrement le + récent
        if "acct_open_date" in df:
            df = df.sort_values("acct_open_date").drop_duplicates(subset=["id"], keep="last")
        else:
            df = df.drop_duplicates(subset=["id"], keep="last")
    return df

def prep_users(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in ["per_capita_income", "yearly_income", "total_debt"]:
        if c in df: df[c] = df[c].apply(clean_currency)
    for c in ["latitude", "longitude"]:
        if c in df: df[c] = pd.to_numeric(df[c], errors="coerce")
    if "address" in df:
        df["address"] = df["address"].apply(normalize_str)
        df["user_city"] = df["address"].apply(infer_city_from_address)
    if "id" in df:
        df["id"] = pd.to_numeric(df["id"], errors="coerce").astype("Int64")
        df = df.drop_duplicates(subset=["id"], keep="last")
    return df

# ============ Chargement ============
transactions_train = pd.read_csv("transactions_train.csv", low_memory=False)
cards_data         = pd.read_csv("cards_data.csv",         low_memory=False)
users_data         = pd.read_csv("users_data.csv",         low_memory=False)

with open("train_fraud_labels.json", "r") as f:
    labels_json = json.load(f)
labels_dict = labels_json.get("target", labels_json)
labels_df = pd.DataFrame(list(labels_dict.items()), columns=["transaction_id", "fraud_label"])

with open("mcc_codes.json", "r") as f:
    mcc_codes = json.load(f)
mcc_df = pd.DataFrame(list(mcc_codes.items()), columns=["mcc", "mcc_description"])

# ============ Nettoyage base ============
transactions_train = prep_transactions(transactions_train)
cards_data         = prep_cards(cards_data)
users_data         = prep_users(users_data)

# Labels
labels_df["transaction_id"] = pd.to_numeric(labels_df["transaction_id"], errors="coerce").astype("Int64")
labels_df["fraud_label"] = labels_df["fraud_label"].map({"Yes": 1, "No": 0}).astype("Int64")

# MCC clé en string
mcc_df["mcc"] = mcc_df["mcc"].astype(str)

# ============ Fusions intelligentes ============
# 1) transactions + labels (garde toutes les transactions)
train_merged = transactions_train.merge(
    labels_df, on="transaction_id", how="left", validate="m:1"
)

# 2) + cartes  (transactions.card_id -> cards.id)
train_merged = train_merged.merge(
    cards_data.add_prefix("card_"),
    left_on="card_id", right_on="card_id",
    how="left", validate="m:1"
)

# 3) + users   (transactions.client_id -> users.id)
train_merged = train_merged.merge(
    users_data.add_prefix("user_"),
    left_on="client_id", right_on="user_id",
    how="left", validate="m:1"
)

# 4) + MCC description
train_merged = train_merged.merge(
    mcc_df, on="mcc", how="left", validate="m:1"
)

# 5) Ville finale : merchant_city prioritaire, sinon user_user_city
if "merchant_city" in train_merged:
    train_merged["city"] = train_merged["merchant_city"].fillna(train_merged.get("user_user_city"))
else:
    train_merged["city"] = train_merged.get("user_user_city")

# 6) Drapeaux qualité utiles
train_merged["has_mcc_desc"]      = train_merged["mcc_description"].notna()
train_merged["is_amount_negative"] = train_merged["amount"] < 0
train_merged["missing_card"]      = train_merged["card_id"].isna()
train_merged["missing_user"]      = train_merged["user_id"].isna()

print("Fusion réussie :", train_merged.shape)

# ============ Option : split stratifié pour modélisation ============
# On split seulement si la cible existe
if "fraud_label" in train_merged and train_merged["fraud_label"].notna().any():
    tmp = train_merged.dropna(subset=["fraud_label"]).copy()
    tmp["fraud_label"] = tmp["fraud_label"].astype(int)
    train_df, valid_df = train_test_split(
        tmp, test_size=0.2, random_state=42, stratify=tmp["fraud_label"]
    )
    print("Split stratifié -> train:", train_df.shape, "valid:", valid_df.shape)
else:
    train_df, valid_df = train_merged, None

# ============ Sauvegardes ============
train_merged.to_csv("merged_train_clean.csv", index=False)
if valid_df is not None:
    train_df.to_csv("train_split.csv", index=False)
    valid_df.to_csv("valid_split.csv", index=False)


/tmp/ipython-input-1125383409.py:83: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  if "acct_open_date" in df: df["acct_open_date"] = pd.to_datetime(df["acct_open_date"], errors="coerce")


Fusion réussie : (210000, 46)
Split stratifié -> train: (168000, 46) valid: (42000, 46)


In [ ]:
train_merged.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,user_yearly_income,user_total_debt,user_credit_score,user_num_credit_cards,user_user_city,mcc_description,has_mcc_desc,is_amount_negative,missing_card,missing_user
0,19765990,2017-07-07 09:53:00,1581,2519,12.35,chip,20519,Arnold,MO,63010,...,44334.0,55173.0,684,5,<NA>,Book Stores,True,False,False,False
1,22160255,2018-11-26 17:44:00,1862,4049,58.29,chip,98648,Des Moines,IA,50313,...,37864.0,57052.0,686,1,<NA>,Fast Food Restaurants,True,False,False,False
2,17566794,2016-03-26 12:42:00,1967,3367,11.03,chip,46978,Lake Forest,CA,92630,...,61793.0,686.0,767,4,<NA>,"Grocery Stores, Supermarkets",True,False,False,False
3,17318690,2016-02-01 08:30:00,921,3457,85.74,chip,63701,Rush,NY,14543,...,54013.0,3628.0,779,8,<NA>,"Grocery Stores, Supermarkets",True,False,False,False
4,20994060,2018-03-24 14:42:00,456,2800,13.43,chip,83271,Estero,FL,33928,...,34947.0,49024.0,751,3,<NA>,Motor Freight Carriers and Trucking,True,False,False,False


In [ ]:
from google.colab import files

# Exemple : ton DataFrame s'appelle merged
train_merged.to_excel("train_merged.xlsx", index=False)

# Téléchargement local
files.download("train_merged.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>